In [1]:
using HierarchicalEOM  # this automatically loads `QuantumToolbox`
using CairoMakie

In [2]:
ϵ = -5
U = 10
σm = sigmam() # σ-
σz = sigmaz() # σz
II = qeye(2)  # identity matrix

# construct the annihilation operator for both spin-up and spin-down
# (utilize Jordan–Wigner transformation)
d_up = tensor(σm, II)
d_dn = tensor(-1 * σz, σm)
Hsys = ϵ * (d_up' * d_up + d_dn' * d_dn) + U * (d_up' * d_up * d_dn' * d_dn)


Quantum Object:   type=Operator   dims=[2, 2]   size=(4, 4)   ishermitian=true
4×4 SparseArrays.SparseMatrixCSC{ComplexF64, Int64} with 2 stored entries:
     ⋅           ⋅           ⋅          ⋅    
     ⋅      -5.0+0.0im       ⋅          ⋅    
     ⋅           ⋅      -5.0+0.0im      ⋅    
     ⋅           ⋅           ⋅          ⋅    

In [3]:
Γ = 2.0 # electron cavity couplings strength
μ = 0 # chimical potential of the fermionic reservoirs
W = 10 # band-widths of the fermionic reservoir
kT = 0.5 # product of the boltzmann constant and temperature
N = 5 # number of the exponential terms for the fermionic reservoir
bath_up = Fermion_Lorentz_Pade(d_up, Γ, μ, W, kT, N)
bath_dn = Fermion_Lorentz_Pade(d_dn, Γ, μ, W, kT, N)
bath_list = [bath_up, bath_dn]
bath_0 = Fermion_Lorentz_Pade(d_dn + d_up, Γ, μ, W, kT, N)
bath_list = [bath_up, bath_dn]

2-element Vector{FermionBath}:
 FermionBath object with 12 terms.

 FermionBath object with 12 terms.


In [4]:
tier = 3
M_even = M_Fermion(Hsys, tier, bath_list)
M_odd = M_Fermion(Hsys, tier, bath_list, ODD)

Preparing block matrices for HEOM Liouvillian superoperator (using 1 threads)...
Progress: [==============================] 100.0% --- Elapsed Time: 0h 00m 04s (ETA: 0h 00m 00s)
Constructing matrix...[DONE]
Preparing block matrices for HEOM Liouvillian superoperator (using 1 threads)...
Progress: [==============================] 100.0% --- Elapsed Time: 0h 00m 00s (ETA: 0h 00m 00s)
Constructing matrix...[DONE]


Fermion type HEOMLS matrix acting on odd-parity ADOs
system dims = [2, 2]
number of ADOs N = 2325
data =
MatrixOperator(37200 × 37200)

In [5]:
ados_s = steadystate(M_even)
ωlist = -20:1:20

Solving steady state for ADOs by linear-solve method...
Calculating left preconditioner with ilu...[DONE]
Solving linear problem...[DONE]


-20:1:20

In [6]:
rho_0 = tensor(basis(2, 0), basis(2, 0))
rho_0 = rho_0 * rho_0'
rho_3 = tensor(basis(2, 1), basis(2, 1))
rho_3 = rho_3 * rho_3'
rho_1 = tensor(basis(2, 0), basis(2, 1))
rho_1 = rho_1 * rho_1'
rho_2 = tensor(basis(2, 1), basis(2, 0))
rho_2 = rho_2 * rho_2'


Quantum Object:   type=Operator   dims=[2, 2]   size=(4, 4)   ishermitian=true
4×4 Matrix{ComplexF64}:
 0.0+0.0im  0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  1.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im  0.0+0.0im

In [7]:
tlist = 0:0.2:50
sol = HEOMsolve1(M_even, rho_0, tlist, e_ops = [rho_0, rho_1, rho_2, rho_3])

Solving time evolution for ADOs by Ordinary Differential Equations method...
Progress: [==============================] 100.0% --- Elapsed Time: 0h 00m 37s (ETA: 0h 00m 00s)


Solution of hierarchical EOM
(return code: Success)
----------------------------
Btier = 0
Ftier = 3
num_ados   = 1
num_expect = 4
ODE alg.: OrdinaryDiffEqLowOrderRK.DP5{typeof(OrdinaryDiffEqCore.trivial_limiter!), typeof(OrdinaryDiffEqCore.trivial_limiter!), Static.False}(OrdinaryDiffEqCore.trivial_limiter!, OrdinaryDiffEqCore.trivial_limiter!, static(false))
abstol = 1.0e-8
reltol = 1.0e-6


In [8]:
# for time evolution
p11_e = real(sol.expect[1, :]) # P00 is the 1st element in e_ops
p22_e = real(sol.expect[2, :]); # P01 is the 3rd element in e_ops
p33_e = real(sol.expect[3, :]);
p44_e = real(sol.expect[4, :]);


#df = DataFrame(rh0_11 = p11_e, rh0_22 = p22_e, rh0_33 = p33_e, rh0_44 = p44_e)
#CSV.write("pop_julia_strong.csv", df)

In [10]:

fig = Figure(size=(500, 350))

ax = Axis(fig[1, 1], 
          xlabel=L"t", 
          ylabel=L"\rho(t)", 
          title="Populations as function of time")

# Création des courbes avec des couleurs différentes et des labels
line1 = lines!(ax, tlist, p11_e, color=:blue, label=L"\rho_{11}")
line2 = lines!(ax, tlist, p22_e, color=:red, label=L"\rho_{22}")
line3 = lines!(ax, tlist, p33_e, color=:green, label=L"\rho_{33}")
line4 = lines!(ax, tlist, p44_e, color=:purple, label=L"\rho_{44}")

# Ajout de la légende
Legend(fig[1, 2], ax, framevisible=true)

fig

save("population_jl.png", fig)